In [59]:
import numpy as np
# Set parameters
sigma = 0.2
r_ = 0
drift = r_ - 0.5 * sigma**2
T = 1
m = 10
delta_t = T / m
N = 10000000 # number of simulations
# Number of RV we simulate is N * m

In [58]:
# Generate paths, note we skip time 0
x = np.linspace(drift * delta_t, drift * T, m)
sims = np.random.normal(0, sigma * T**0.5, (N, m)) # note scale param is sd not var
y = x + sims

res = np.any(y > 0, axis=1)

prob = np.sum(~res) / N
sd = (prob * (1 - prob) / N)**0.5
print(f"Probability of maximum 0 is about {prob}, with sd {sd}")
print(f"Hence 95% CI: [{prob - 1.96 * sd}, {prob + 1.96 * sd}")

Probability of maximum 0 is about 0.0014952, with sd 1.2218692143433356e-05
Hence 95% CI: [0.0014712513633988705, 0.0015191486366011293


In [ ]:
# Now we will estimate B2 of the paper Broadie 1999
sigma = 1
T = 1
m = 100
M = 1000 # this is BM path, above is discrete
N = 1
drift = r_ - 0.5 * sigma**2
delta_t = T = M

BM_path = np.linspace(drift * delta_t, drift * T, M)
